In [17]:
import cv2 as cv
import open3d as o3d
import numpy as np

In [18]:
def o3d_to_cv(pointcloud):
    if pointcloud.has_normals() and pointcloud.has_points():
        return(np.concatenate((np.asarray(pointcloud.points), np.asarray(pointcloud.normals)), axis=1, dtype=np.float32))
    else:
        print("incorrect input format")

## Preprocess point clouds

In [19]:
#pcd = o3d.io.read_point_cloud("data/monkey100.ply")
mesh = o3d.io.read_triangle_mesh("data/monkey.ply")
pcd = mesh.sample_points_uniformly(number_of_points=2000)

scene_pcd = o3d.io.read_point_cloud("data/_frames_1_to_8.ply")

pt_plane_distance = 0.02

plane_model, inliers = scene_pcd.segment_plane(distance_threshold=pt_plane_distance, ransac_n=3, num_iterations=1000)

inlier_cloud = scene_pcd.select_by_index(inliers)
outlier_cloud = scene_pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([0.6,0.6,0.6])

#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

scene_pcd = outlier_cloud

pcd = pcd.translate([-10,-10,1])

down_scene_pcd = scene_pcd.voxel_down_sample(voxel_size=0.05)

#o3d.visualization.draw_geometries([down_scene_pcd100])

nn_distance = np.mean(down_scene_pcd.compute_nearest_neighbor_distance())
normal_radius = nn_distance*4

down_scene_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius,max_nn=16))

down_scene_pcd.orient_normals_consistent_tangent_plane(k=16)

pcd_formatted = o3d_to_cv(pcd)
scene_formatted = o3d_to_cv(down_scene_pcd)

In [ ]:
o3d.visualization.draw_geometries([down_scene_pcd, pcd], point_show_normal=True)

In [21]:
N = 2

detector = cv.ppf_match_3d_PPF3DDetector(0.025, 0.05)
print('Loading model...')

print('Training...')
detector.trainModel(pcd_formatted)

Loading model...
Training...


In [22]:
print('Matching...')
results = detector.match(scene_formatted, 0.5, 0.05)

print('Performing ICP...')
icp = cv.ppf_match_3d_ICP(100)
_, results = icp.registerModelToScene(pcd_formatted, scene_formatted, results[:N])

print("Poses: ")
for i, result in enumerate(results):
    #result.printPose()
    print("\n-- Pose to Model Index %d: NumVotes = %d, Residual = %f\n%s\n" % (result.modelIndex, result.numVotes, result.residual, result.pose))
    if i == 0:
        pct = cv.ppf_match_3d.transformPCPose(pcd_formatted, result.pose)
        cv.ppf_match_3d.writePLY(pct, "%sPCTrans.ply" % "modelname")

Matching...
Performing ICP...
Poses: 

-- Pose to Model Index 1014: NumVotes = 361, Residual = 0.008462
[[ 9.99913975e-01 -1.17549685e-02 -5.81921384e-03  9.91719164e+00]
 [ 1.17715627e-02  9.99926720e-01  2.82564292e-03  1.01301934e+01]
 [ 5.78557206e-03 -2.89390109e-03  9.99979076e-01 -9.58574104e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


-- Pose to Model Index 1147: NumVotes = 297, Residual = 0.008241
[[ 9.99935134e-01 -1.00037201e-02 -5.44542593e-03  9.93030483e+00]
 [ 1.00148618e-02  9.99947804e-01  2.02265856e-03  1.01122077e+01]
 [ 5.42490760e-03 -2.07706255e-03  9.99983128e-01 -9.56204026e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [23]:
translate_pcd = o3d.io.read_point_cloud("./modelnamePCTrans.ply")
scene_pcd.paint_uniform_color([0.6,0.6,0.6])
o3d.visualization.draw_geometries([scene_pcd, translate_pcd])

# Match segment to position

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [106]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(scene_pcd.cluster_dbscan(eps=0.5, min_points=10, print_progress=True))

clusters = []

for i in range(labels.max()+1):
    clusters.append(scene_pcd.select_by_index(np.where(labels == i)[0]))

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.                     ] 2%
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2
Precompute neighbors.[========================================] 100%======================>        ] 77%
Clustering[========================================] 100%


### Distance meassure approach

### KNN approach

In [108]:
KNN = KNeighborsClassifier()

KNN.fit(X=np.asarray(scene_pcd.points), y=labels)

select = int(np.median(KNN.predict(np.asarray(translate_pcd.points))))
o3d.visualization.draw_geometries([clusters[select], translate_pcd])

In [112]:
hull, _ = clusters[select].compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)

o3d.visualization.draw_geometries([clusters[select], mesh])

In [113]:
tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(clusters[0])

In [116]:
alpha = 0.1

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(clusters[0], alpha, tetra_mesh, pt_map)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])